In [22]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
plt.style.use ('ggplot')
import seaborn as sns
%matplotlib inline
sns.set_style(style="whitegrid")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text

from bs4 import BeautifulSoup

import nltk, string, contractions, random
from nltk.stem.snowball import SnowballStemmer

from gensim.corpora.dictionary import Dictionary
from gensim.models.nmf import Nmf
from gensim.models.coherencemodel import CoherenceModel

from operator import itemgetter

In [23]:
df1 = pd.read_csv('data/modeling_ready_microwave1')



In [24]:
df1.dropna(inplace=True)



In [25]:
df1.review_date = pd.to_datetime(df1.review_date)



In [26]:
keep_words = ['not', 'ain', 'aren', "aren't", 'couldn', "couldn't", 
              'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 
              'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 
              'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn',
              "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 
              'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 
              'wouldn', "wouldn't", 'fire', 'off']
test_stop_words = []
review_stop_words = ['panasonic', 'really', 'husband', 'thanks', 'thank', 'ha', 
              'just', 'thing', 'did', 'nn', 'wa', 'yr', 'u', 'say', 'doe',
              'mom', 'christmas', 'gift', 'got', 'way', 'le', 'daughter',
              'e','not','love','good','bought','great microwave','micro',
              'great oven','microwave','product','work great','nice work',
              'work great use','work great love','feature work great',
              'unit work great','oven work great','easy use love','old oven',
              'old old','20 year old','unit', 'not', 'work', 'amazon', 'com',
              'old', 'wife', 'highly', 'recommend', 'like', 'charm', '20', 
              'easy', 'oven', 'use', 'year', 'lot', 'pleased', 'happy', 'hope',
              'review', 'buy', 'far', 'day', '1', '2', '3', '4', '5', '6', '7',
              '8', '9', '10', '20', 'great', 'excellent','feature', 'nice', 'old']
for _ in text.ENGLISH_STOP_WORDS:
    if _ in keep_words:
        pass
    else:
        review_stop_words.append(_)
for _ in test_stop_words:
    review_stop_words.append(_)   
    
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()    
    
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

def vectorize_this(df, max_features, min_df, max_df, ngram_max):
    vectorizer = TfidfVectorizer(tokenizer = lemmatize_text,
                             stop_words= review_stop_words,
                             max_features = max_features,
                             min_df = min_df,
                             max_df =  max_df,
                             ngram_range=(1, ngram_max)
                            )
    X = df['model_ready']
    X = vectorizer.fit_transform(X)
    return X, vectorizer

In [27]:
topics1 = 7
mindf1 = 2
maxdf1 = .99
grams1 = 3

In [ ]:
# topics2 = 7
# mindf2 = 2
# maxdf2 = .99
# grams2 = 3

In [ ]:
# topics3 = 7
# mindf3 = 2
# maxdf3 = .99
# grams3 = 3

In [28]:
X1, vectorizer1 = vectorize_this(df1, 10000, mindf, maxdf, grams)

In [ ]:
# X2, vectorizer2 = vectorize_this(df2, 10000, mindf, maxdf, grams)

In [ ]:
# X3, vectorizer3 = vectorize_this(df3, 10000, mindf, maxdf, grams)

In [29]:
def make_nmf(n_components, alpha, X):
    nmf = NMF(
            n_components=n_components,
            init='nndsvd',
            random_state=12345,
            alpha = alpha
            ).fit(X)

    W = nmf.fit_transform(X)
    H = nmf.components_
    return nmf, W, H

In [30]:
nmf1, W1, H1, = make_nmf(topics, .01, X1)

In [ ]:
# nmf2, W2, H2, = make_nmf(topics, .1, X2)

In [ ]:
# nmf3, W3, H3, = make_nmf(topics, .1, X3)

In [33]:
def topic_keywords(lda_model, vectorizer=vectorizer, n_words=10):
    
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

In [34]:
topic_keywords1 = topic_keywords(nmf1, vectorizer1)        

In [ ]:
# topic_keywords2 = topic_keywords(vectorizer2, nmf2)        

In [ ]:
# topic_keywords3 = topic_keywords(vectorizer3, nmf3)        

In [35]:
topic_keywords1

[array(['time', 'popcorn', 'button', 'cooking', 'setting', 'minute', 'bag',
        'used', 'second', 'long'], dtype='<U31'),
 array(['door', 'open', 'latch', 'close', 'door latch', 'push', 'button',
        'door open', 'problem', 'hard'], dtype='<U31'),
 array(['lasted', 'month', 'working', 'warranty', 'service', 'repair',
        'died', 'new', 'model', 'purchased'], dtype='<U31'),
 array(['price', 'large', 'fit', 'size', 'big', 'space', 'counter', 'need',
        'kitchen', 'needed'], dtype='<U31'),
 array(['power', 'level', 'power level', 'watt', 'inverter', '1250',
        '1250 watt', 'setting', 'power setting', 'watt power'],
       dtype='<U31'),
 array(['sensor', 'reheat', 'sensor reheat', 'food', 'defrost', 'function',
        'button', 'reheat function', 'sensor cook', 'used'], dtype='<U31'),
 array(['cook', 'food', 'heat', 'evenly', 'fast', 'quickly', 'cook food',
        'cook fast', 'heat food', 'powerful'], dtype='<U31')]

In [36]:
# def topic_featuring(X, topic_keywords, n_components=10, n_words=10):
    
#     nmf, W, H = make_nmf(n_components, .1, X)
    
#     # Topic - Keywords Dataframe
#     df_topic_keywords = pd.DataFrame(topic_keywords)
#     df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
#     df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

#     Topics_theme = range(n_components)
#     df_topic_keywords['topic_theme'] = Topics_theme
#     df_topic_keywords.set_index('topic_theme', inplace=True)
#     return df_topic_keywords.T

In [37]:
# topic_featuring(X1, topic_keywords1, topics1)

topic_theme,0,1,2,3,4,5,6
Word 0,time,door,lasted,price,power,sensor,cook
Word 1,popcorn,open,month,large,level,reheat,food
Word 2,button,latch,working,fit,power level,sensor reheat,heat
Word 3,cooking,close,warranty,size,watt,food,evenly
Word 4,setting,door latch,service,big,inverter,defrost,fast
Word 5,minute,push,repair,space,1250,function,quickly
Word 6,bag,button,died,counter,1250 watt,button,cook food
Word 7,used,door open,new,need,setting,reheat function,cook fast
Word 8,second,problem,model,kitchen,power setting,sensor cook,heat food
Word 9,long,hard,purchased,needed,watt power,used,powerful


In [ ]:
# topic_featuring(X2, topic_keywords2, topics2)

In [ ]:
# topic_featuring(X3, topic_keywords3, topics3)

In [55]:
def df_featurizer(df, topic_keywords, topic_labels=None, n_components=10, column_names_known = 'n'):

    # Topic - Keywords Dataframe
    df_topic_keywords = pd.DataFrame(topic_keywords)
    df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
    df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
    
    if column_names_known == 'n':
        Topics_theme = range(n_components)
    elif column_names_known == 'Y':
        Topics_theme = topic_labels
    df_topic_keywords['topic_theme'] = Topics_theme
    df_topic_keywords.set_index('topic_theme', inplace=True)
    return df_topic_keywords.T

In [56]:
df_featurizer(df1, topic_keywords1, n_components=topics1)

topic_theme,0,1,2,3,4,5,6
Word 0,time,door,lasted,price,power,sensor,cook
Word 1,popcorn,open,month,large,level,reheat,food
Word 2,button,latch,working,fit,power level,sensor reheat,heat
Word 3,cooking,close,warranty,size,watt,food,evenly
Word 4,setting,door latch,service,big,inverter,defrost,fast
Word 5,minute,push,repair,space,1250,function,quickly
Word 6,bag,button,died,counter,1250 watt,button,cook food
Word 7,used,door open,new,need,setting,reheat function,cook fast
Word 8,second,problem,model,kitchen,power setting,sensor cook,heat food
Word 9,long,hard,purchased,needed,watt power,used,powerful


In [ ]:
# nmf_featurizer(df2, max_df=maxdf2, ngram_max=grams2, n_components=topics2)

In [ ]:
# nmf_featurizer(df3, max_df=maxdf3, ngram_max=grams3, n_components=topics3)

In [45]:
vocabulary1 = np.array(vectorizer1.get_feature_names())

In [ ]:
# vocabulary2 = np.array(vectorizer2.get_feature_names())

In [ ]:
# vocabulary3 = np.array(vectorizer3.get_feature_names())

In [46]:
def label_topics(H, vocabulary):
    '''
    Print the most influential words of each latent topic, and prompt the user
    to label each topic. The user should use their humanness to figure out what
    each latent topic is capturing.
    '''
    topic_labels = []
    for i, row in enumerate(H):
        top_five = np.argsort(row)[::-1][:12]
        print('topic', i)
        print('-->', ' '.join(vocabulary[top_five]))
        label = input('please label this topic: ')
        topic_labels.append(label)
        print()
    return topic_labels

In [62]:
topic_labels1 = label_topics(H1,vocabulary1)

topic 0
--> time popcorn button cooking setting minute bag used second long pop popcorn setting
please label this topic: popcorn

topic 1
--> door open latch close door latch push button door open problem hard open door slam
please label this topic: door

topic 2
--> lasted month working warranty service repair died new model purchased worked stopped
please label this topic: short lifespan

topic 3
--> price large fit size big space counter need kitchen needed look wanted
please label this topic: size

topic 4
--> power level power level watt inverter 1250 1250 watt setting power setting watt power cooking technology
please label this topic: power

topic 5
--> sensor reheat sensor reheat food defrost function button reheat function sensor cook used sensor reheat function model
please label this topic: programmed functions

topic 6
--> cook food heat evenly fast quickly cook food cook fast heat food powerful cook evenly food quickly
please label this topic: cooks fast and evenly



In [ ]:
# topic_labels2 = label_topics(H2,vocabulary2)

In [ ]:
# topic_labels3 = label_topics(H3,vocabulary3)

In [63]:
df_featurizer(df1, topic_keywords1, topic_labels1, n_components=topics1, column_names_known = 'Y')

topic_theme,popcorn,door,short lifespan,size,power,programmed functions,cooks fast and evenly
Word 0,time,door,lasted,price,power,sensor,cook
Word 1,popcorn,open,month,large,level,reheat,food
Word 2,button,latch,working,fit,power level,sensor reheat,heat
Word 3,cooking,close,warranty,size,watt,food,evenly
Word 4,setting,door latch,service,big,inverter,defrost,fast
Word 5,minute,push,repair,space,1250,function,quickly
Word 6,bag,button,died,counter,1250 watt,button,cook food
Word 7,used,door open,new,need,setting,reheat function,cook fast
Word 8,second,problem,model,kitchen,power setting,sensor cook,heat food
Word 9,long,hard,purchased,needed,watt power,used,powerful


In [ ]:
# df_featurizer(df2, topic_keywords2, topic_labels2, n_components=topics2, column_names_known = 'Y')

In [ ]:
# df_featurizer(df3, topic_keywords3, topic_labels3, n_components=topics3, column_names_known = 'Y')

In [64]:
def softmax(v, temperature=1.0):
    '''
    A heuristic to convert arbitrary positive values into probabilities.
    See: https://en.wikipedia.org/wiki/Softmax_function
    '''
    expv = np.exp(v / temperature)
    s = np.sum(expv)
    return expv / s

In [65]:
def analyze_reviews(W, topic_labels):
    '''
    Print an analysis of a single Amazon review, including the review ID
    and a summary of which topics it represents. The topics are identified
    via the hand-labels which were assigned by the user.
    '''
    topic_count = len(topic_labels)
    topic_percentages = [[] for i in range(0, topic_count)]

    for idx, i in enumerate(W):  
        probs = softmax(W[idx], temperature=0.01)
        topic_list_counter = 0
        for prob, label in zip(probs, topic_labels):
            topic_percentages[topic_list_counter].append(round(prob, 5))
            topic_list_counter += 1
        
    return pd.DataFrame(dict(zip(topic_labels, topic_percentages)))

In [66]:
top_df1 = analyze_reviews(W1, topic_labels1)

In [ ]:
# top_df2 = analyze_reviews(W2, topic_labels2)

In [ ]:
# top_df3 = analyze_reviews(W3, topic_labels3)

In [67]:
top_df1

,popcorn,door,short lifespan,size,power,programmed functions,cooks fast and evenly
0,0.00010,0.99988,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.99999
2,0.00013,0.00013,0.00013,0.99912,0.00013,0.00013,0.00021
3,0.01344,0.01287,0.91099,0.02409,0.01287,0.01287,0.01287
4,0.00000,0.99998,0.00000,0.00000,0.00000,0.00000,0.00001
...,...,...,...,...,...,...,...
1843,0.20434,0.00382,0.00382,0.67517,0.00426,0.00856,0.10002
1844,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000
1845,0.01104,0.00005,0.00005,0.00534,0.00005,0.00005,0.98342
1846,0.99891,0.00015,0.00034,0.00015,0.00015,0.00015,0.00016


In [68]:
top_df1.loc[0]

popcorn                  0.00010
door                     0.99988
short lifespan           0.00000
size                     0.00000
power                    0.00000
programmed functions     0.00000
cooks fast and evenly    0.00000
Name: 0, dtype: float64

In [74]:
def feature_score(top_df):    
    feature_score = {}
    for _ in range(len(top_df.columns)):
        key = top_df.columns[_]
        feature_score[key] = round(sum(top_df[key])/(len(top_df)),5)
    return feature_score

In [77]:
feature_score1 = feature_score(top_df1)
sorted(feature_score1.items(), key= lambda x: x[1], reverse=True)

[('size', 0.21387),
 ('short lifespan', 0.20912),
 ('door', 0.13818),
 ('cooks fast and evenly', 0.11958),
 ('popcorn', 0.11585),
 ('programmed functions', 0.10639),
 ('power', 0.09702)]

In [ ]:
# feature_score2 = feature_score(top_df2)
# sorted(feature_score2.items(), key= lambda x: x[1], reverse=True)

In [ ]:
# feature_score3 = feature_score(top_df3)
# sorted(feature_score3.items(), key= lambda x: x[1], reverse=True)

In [80]:
topic_words_df1 = nmf_featurizer(df1, topic_keywords1, topic_labels1, column_names_known ='Y')
# topic_words_df2 = nmf_featurizer(df2, topic_keywords2, topic_labels2, column_names_known ='Y')
# topic_words_df3 = nmf_featurizer(df3, topic_keywords3, topic_labels3, column_names_known ='Y')


topic_theme,popcorn,door,short lifespan,size,power,programmed functions,cooks fast and evenly
Word 0,time,door,lasted,price,power,sensor,cook
Word 1,popcorn,open,month,large,level,reheat,food
Word 2,button,latch,working,fit,power level,sensor reheat,heat
Word 3,cooking,close,warranty,size,watt,food,evenly
Word 4,setting,door latch,service,big,inverter,defrost,fast
Word 5,minute,push,repair,space,1250,function,quickly
Word 6,bag,button,died,counter,1250 watt,button,cook food
Word 7,used,door open,new,need,setting,reheat function,cook fast
Word 8,second,problem,model,kitchen,power setting,sensor cook,heat food
Word 9,long,hard,purchased,needed,watt power,used,powerful


In [98]:
review_df_columns1 = (['review_body'] + topic_labels1)
# review_df_columns2 = (['review_body'] + topic_labels2)
# review_df_columns3 = (['review_body'] + topic_labels3)

In [85]:
review_df1 = pd.concat([df1, top_df1.reindex(df1.index)], axis=1)[review_df_columns1]
# review_df2 = pd.concat([df2, top_df2.reindex(df2.index)], axis=1)[review_df_columns2]
# review_df3 = pd.concat([df3, top_df3.reindex(df3.index)], axis=1)[review_df_columns3]

In [94]:
def review_checker(review_df, num_samples):
    review_df = review_df
    review_index = []
    for _ in range(num_samples):
        review_index.append(random.randint(0, len(review_df)))
        
    for _ in review_index:
        print('Review ' + str(_))
        print(' ')
        print(review_df.review_body[_])
        print(' ')
        print(review_df.loc[_][1:])
        print(' ')
        print('___________________________________________________')
        print(' ')
    

In [97]:
review_checker(review_df1, 5)

Review 27
 
Lasted 17 months before door interlock failed (turns off power).  YouTube shows repair but needs special tool and I was surprized at how flimsy the assembly appeared in the video.  The over-stiff latch always closed the door with a loud kathunk!  My previous Panasonic microwave was great and lasted 20 years of constant use and my many other products made by Panasonic had been excellent.  What a waste of money and a disappointment! Panasonic's past high quality approach seems to be absent in the Panasonic NN-H965BF Genius 2.2 cuft 1250 Watt microwave oven..
 
popcorn                  0.01863
door                     0.01655
short lifespan           0.91394
size                       0.019
power                    0.01071
programmed functions     0.01059
cooks fast and evenly    0.01059
Name: 27, dtype: object
 
___________________________________________________
 
Review 488
 
made in china .....it is almost like we are forced to buy merchandise foreign made...I would rather